In [ ]:
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import messagebox, scrolledtext, LabelFrame
import os
import re
from textblob import TextBlob
import matplotlib.pyplot as plt
import nltk
import io
from collections import Counter
import mysql.connector
from mysql.connector import Error
import hashlib
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image as RLImage, Table, TableStyle
from reportlab.lib import colors
from tkinter import filedialog
import tempfile
import datetime

# Download required NLTK data
try:
    nltk.download('punkt', quiet=True)
except:
    pass

class DatabaseManager:
    def __init__(self):
        self.connection = None
        self.connect_to_database()
    
    def connect_to_database(self):
        """Connect to MySQL database"""
        try:
            self.connection = mysql.connector.connect(
                host='localhost',  # Change if your MySQL server is on a different host
                database='project',  # Your database name
                user='root',  # Change to your MySQL username
                password='saini800327'  # Change to your MySQL password
            )
            
            if self.connection.is_connected():
                print("Successfully connected to MySQL database")
                self.create_users_table()
                
        except Error as e:
            print(f"Error connecting to MySQL: {e}")
            messagebox.showerror("Database Error", f"Failed to connect to database: {e}")
    
    def create_users_table(self):
        """Create users table if it doesn't exist"""
        try:
            cursor = self.connection.cursor()
            create_table_query = """
            CREATE TABLE IF NOT EXISTS users (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(100) NOT NULL,
                email VARCHAR(100) UNIQUE NOT NULL,
                username VARCHAR(50) UNIQUE NOT NULL,
                password VARCHAR(255) NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
            """
            cursor.execute(create_table_query)
            self.connection.commit()
            cursor.close()
            print("Users table created successfully")
            
        except Error as e:
            print(f"Error creating table: {e}")
    
    def hash_password(self, password):
        """Hash password using SHA-256"""
        return hashlib.sha256(password.encode()).hexdigest()
    
    def register_user(self, name, email, username, password):
        """Register a new user in the database"""
        try:
            cursor = self.connection.cursor()
            hashed_password = self.hash_password(password)
            
            # Check if username or email already exists
            check_query = "SELECT username, email FROM users WHERE username = %s OR email = %s"
            cursor.execute(check_query, (username, email))
            existing_user = cursor.fetchone()
            
            if existing_user:
                cursor.close()
                return False, "Username or email already exists"
            
            # Insert new user
            insert_query = """
            INSERT INTO users (name, email, username, password) 
            VALUES (%s, %s, %s, %s)
            """
            cursor.execute(insert_query, (name, email, username, hashed_password))
            self.connection.commit()
            cursor.close()
            
            return True, "User registered successfully"
            
        except Error as e:
            print(f"Error registering user: {e}")
            return False, f"Registration failed: {e}"
    
    def authenticate_user(self, username, password):
        """Authenticate user login"""
        try:
            cursor = self.connection.cursor()
            hashed_password = self.hash_password(password)
            
            query = "SELECT id, name FROM users WHERE username = %s AND password = %s"
            cursor.execute(query, (username, hashed_password))
            user = cursor.fetchone()
            cursor.close()
            
            if user:
                return True, f"Welcome back, {user[1]}!"
            else:
                return False, "Invalid username or password"
                
        except Error as e:
            print(f"Error authenticating user: {e}")
            return False, f"Authentication failed: {e}"
    
    def close_connection(self):
        """Close database connection"""
        if self.connection and self.connection.is_connected():
            self.connection.close()
            print("MySQL connection closed")

class TextAnalyzerApp:
    def __init__(self):
        self.root = None
        self.current_page = None
        self.db_manager = DatabaseManager()
        self.analysis_results = None  # Initialize analysis results
        
    def start_app(self):
        """Start with the getting started page"""
        self.show_getting_started()
    
    def clear_window(self):
        """Clear all widgets from current window"""
        if self.root:
            for widget in self.root.winfo_children():
                widget.destroy()
    
    def show_getting_started(self):
        """Show the getting started page"""
        if self.root:
            self.root.destroy()
            
        self.root = tk.Tk()
        self.root.title("TextAnalyzer")
        self.root.configure(bg='white')
        self.root.geometry("400x600")
        self.current_page = "getting_started"

        frame = tk.Frame(self.root, bg="white", bd=0, relief="groove", padx=20, pady=20, height=400)
        frame.place(relx=0.5, rely=0.5, anchor="center")

        title = tk.Label(frame, text="TextAnalyzer", font=("Helvetica", 18, "bold"), fg='black', bg="white")
        title.pack(pady=(10, 5))

        subtitle = tk.Label(frame, text="Enhance your text analysis skills.", font=("Helvetica", 13), bg="white")
        subtitle.pack(pady=(0, 10))

        # Load image if available
        image_path = "avatarrr.png"
        if os.path.exists(image_path):
            try:
                avatar_image = Image.open(image_path).resize((200, 200))
                avatar_tk = ImageTk.PhotoImage(avatar_image)
                img_label = tk.Label(frame, image=avatar_tk, bg="white", bd=2, relief="solid")
                img_label.image = avatar_tk
                img_label.pack(pady=10)
            except Exception as e:
                print("Image error:", e)

        btn = tk.Button(
            frame,
            text="Get Started",
            bg="#87CEFA",
            fg="white",
            font=("Helvetica", 13, "bold"),
            relief="flat",
            bd=0,
            padx=20,
            pady=5,
            command=self.on_get_started
        )
        btn.pack(pady=20)

        self.root.mainloop()
    
    def on_get_started(self):
        """Handle get started button click"""
        response = messagebox.askyesno("Confirmation", "Do you want to get started?")
        if response:
            self.show_login_page()
        else:
            self.db_manager.close_connection()
            self.root.destroy()
    
    def show_login_page(self):
        """Show the login page"""
        self.clear_window()
        self.root.title("Login Page")
        self.root.geometry("900x500")
        self.root.configure(bg='white')
        self.root.resizable(False, False)
        self.current_page = "login"

        # Left Frame with Image
        left_frame = tk.Frame(self.root, bg="white", width=650)
        left_frame.pack(side="left", fill="both")

        # Load and show image
        image_path = "loginwordtok.jpg"
        if os.path.exists(image_path):
            try:
                img = Image.open(image_path)
                img = img.resize((550, 500))
                img_tk = ImageTk.PhotoImage(img)
                img_label = tk.Label(left_frame, image=img_tk, bg="white")
                img_label.image = img_tk
                img_label.pack(fill="both", expand=True)
            except Exception as e:
                print("Image error:", e)

        # Right Frame with Login Form
        right_frame = tk.Frame(self.root, bg="white", width=450)
        right_frame.pack(side="right", fill="both", expand=True)

        # Welcome title
        label_title = tk.Label(right_frame, text="Log in", font=("Helvetica", 20, "bold"), fg='#25ccf4', bg='white')
        label_title.pack(pady=20)
        label_subtitle = tk.Label(right_frame, text="Access Advance Text Analysis features Now", font=("Helvetica", 10, "bold"), bg='white')
        label_subtitle.pack(pady=20)

        # Username input
        label_username = tk.Label(right_frame, text="Username", font=("Arial", 13, "bold"), fg='black', bg='white')
        label_username.pack(anchor='w', padx=40)
        self.entry_username = tk.Entry(right_frame, font=("Arial", 14), bd=0)
        self.entry_username.pack(padx=40, fill='x')
        self.add_placeholder(self.entry_username, "Enter username")

        tk.Frame(right_frame, height=3, bg="black").pack(padx=40, fill='x', pady=(0, 20))

        # Password input
        label_password = tk.Label(right_frame, text="Password", font=("Arial", 13, "bold"), fg='black', bg='white')
        label_password.pack(anchor='w', padx=40)
        self.entry_password = tk.Entry(right_frame, font=("Arial", 14), bd=0, show="*")
        self.entry_password.pack(padx=40, fill='x')
        self.add_placeholder(self.entry_password, "Enter password", is_password=True)

        tk.Frame(right_frame, height=3, bg="black").pack(padx=40, fill='x', pady=(0, 20))

        # Login Button
        btn_login = tk.Button(right_frame, text="Login", font=("Arial", 12), bg='#25ccf4', fg='white', width=20, bd=0, command=self.validate_login)
        btn_login.pack(pady=10)

        # Social buttons
        social_frame = tk.Frame(right_frame, bg="white")
        social_frame.pack(pady=10)

        for icon in ["G", "f", "t"]:
            btn = tk.Label(social_frame, text=icon, font=("Arial", 12, "bold"), bg="#e1effa",
                          fg="black", width=4, height=2, bd=0, relief="ridge", padx=5, pady=5)
            btn.pack(side='left', padx=10)

        # Sign-up prompt
        signup_frame = tk.Frame(right_frame, bg="white")
        signup_frame.pack(pady=10)

        tk.Label(signup_frame, text="New user? Create an account. ", bg="white", fg="gray", font=("Arial", 9)).pack(side="left")
        signup_link = tk.Label(signup_frame, text="Sign up", bg="white", fg="black", font=("Arial", 9, "bold"), cursor="hand2")
        signup_link.pack(side="left")
        signup_link.bind("<Button-1>", lambda e: self.show_signup_page())
    
    def show_signup_page(self):
        """Show the signup page"""
        self.clear_window()
        self.root.title("Sign Up")
        self.root.geometry("900x600")
        self.root.configure(bg='white')
        self.current_page = "signup"

        # Left Frame (Image)
        left_frame = tk.Frame(self.root, width=400, bg='white')
        left_frame.pack(side='left', fill='both', expand=True)

        # Load image
        image_path = "signup.jpeg"
        if os.path.exists(image_path):
            try:
                img = Image.open(image_path)
                img = img.resize((450, 650))
                img_tk = ImageTk.PhotoImage(img)
                img_label = tk.Label(left_frame, image=img_tk, bg="white")
                img_label.image = img_tk
                img_label.pack(fill="both", expand=True)
            except Exception as e:
                print("Image error:", e)

        # Right Frame (Form)
        right_frame = tk.Frame(self.root, padx=30, pady=30, bg='white')
        right_frame.pack(side='right', fill='both', expand=True)

        tk.Label(right_frame, text="Sign up", font=("Arial", 24, "bold"), bg='white').pack(anchor='w')
        tk.Label(right_frame, text="Start using advanced text analysis features now.", font=("Arial", 10), bg='white', fg='gray').pack(anchor='w', pady=(0, 20))

        # Name
        tk.Label(right_frame, text="Your complete name", font=("Arial", 13, "bold"), fg='black', bg='white').pack(anchor='w', padx=40)
        self.name_entry = tk.Entry(right_frame, font=("Arial", 14), bd=0)
        self.name_entry.pack(padx=40, fill='x')
        self.add_placeholder(self.name_entry, "Enter your full name")
        tk.Frame(right_frame, height=3, bg="black").pack(padx=40, fill='x', pady=(0, 20))

        # Email
        tk.Label(right_frame, text="Email", font=("Arial", 13, "bold"), fg='black', bg='white').pack(anchor='w', padx=40)
        self.email_entry = tk.Entry(right_frame, font=("Arial", 14), bd=0)
        self.email_entry.pack(padx=40, fill='x')
        self.add_placeholder(self.email_entry, "example@email.com")
        tk.Frame(right_frame, height=3, bg="black").pack(padx=40, fill='x', pady=(0, 20))

        # Username
        tk.Label(right_frame, text="Username", font=("Arial", 13, "bold"), fg='black', bg='white').pack(anchor='w', padx=40)
        self.username_entry = tk.Entry(right_frame, font=("Arial", 14), bd=0)
        self.username_entry.pack(padx=40, fill='x')
        self.add_placeholder(self.username_entry, "Choose a username")
        tk.Frame(right_frame, height=3, bg="black").pack(padx=40, fill='x', pady=(0, 20))

        # Password
        tk.Label(right_frame, text="Set up your secure password", font=("Arial", 13, "bold"), fg='black', bg='white').pack(anchor='w', padx=40)
        self.password_entry = tk.Entry(right_frame, font=("Arial", 14), bd=0, show="*")
        self.password_entry.pack(padx=40, fill='x')
        self.add_placeholder(self.password_entry, "Enter password", is_password=True)
        tk.Frame(right_frame, height=3, bg="black").pack(padx=40, fill='x', pady=(0, 20))

        # Confirm Password
        tk.Label(right_frame, text="Confirm password", font=("Arial", 13, "bold"), fg='black', bg='white').pack(anchor='w', padx=40)
        self.confirm_entry = tk.Entry(right_frame, font=("Arial", 14), bd=0, show="*")
        self.confirm_entry.pack(padx=40, fill='x')
        self.add_placeholder(self.confirm_entry, "Confirm password", is_password=True)
        tk.Frame(right_frame, height=3, bg="black").pack(padx=40, fill='x', pady=(0, 20))

        # Terms and Conditions
        self.terms_var = tk.IntVar()
        terms_check = tk.Checkbutton(right_frame, text="By signing up, you agree to our Terms and Privacy Policy.", 
                                    variable=self.terms_var, fg='grey', bg='white', font=("Arial", 9))
        terms_check.pack(anchor='w', padx=40, pady=(15, 10))

        # Submit Button
        submit_btn = tk.Button(right_frame, text="Join now", bg='#6495ED', fg='white', width=30, height=2, command=self.signup)
        submit_btn.pack(padx=40, pady=10)

        # Login Redirect
        login_link = tk.Label(right_frame, text="Already a member? Log in", bg='white', fg='gray', cursor="hand2")
        login_link.pack(anchor='w', padx=40, pady=(10, 0))
        login_link.bind("<Button-1>", lambda e: self.show_login_page())
    
    def add_placeholder(self, entry, placeholder, is_password=False):
        """Add placeholder behavior to entry widgets"""
        def on_focus_in(event):
            if entry.get() == placeholder:
                entry.delete(0, tk.END)
                entry.config(fg='black', show="*" if is_password else '')

        def on_focus_out(event):
            if entry.get() == '':
                entry.insert(0, placeholder)
                entry.config(fg='gray', show='' if is_password else '')

        entry.insert(0, placeholder)
        entry.config(fg='gray')
        entry.bind("<FocusIn>", on_focus_in)
        entry.bind("<FocusOut>", on_focus_out)
    
    def validate_login(self):
        """Validate login credentials against database"""
        username = self.entry_username.get().strip()
        password = self.entry_password.get()

        # Check for placeholder values or empty fields
        if not username or username == "Enter username":
            messagebox.showerror("Error", "Please enter a valid username.")
            return

        if not password or password == "Enter password":
            messagebox.showerror("Error", "Please enter a valid password.")
            return

        # Authenticate with database
        success, message = self.db_manager.authenticate_user(username, password)
        
        if success:
            messagebox.showinfo("Login Success", message)
            self.show_text_analyzer()
        else:
            messagebox.showerror("Login Failed", message)
    
    def signup(self):
        """Handle signup logic with database storage"""
        name = self.name_entry.get().strip()
        email = self.email_entry.get().strip()
        username = self.username_entry.get().strip()
        password = self.password_entry.get()
        confirm = self.confirm_entry.get()
        agree = self.terms_var.get()

        # Check for placeholder values
        if (not name or name == "Enter your full name" or
            not email or email == "example@email.com" or
            not username or username == "Choose a username" or
            not password or password == "Enter password" or
            not confirm or confirm == "Confirm password"):
            messagebox.showerror("Error", "Please fill in all fields.")
            return
        
        # Validate email format
        email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        if not re.match(email_pattern, email):
            messagebox.showerror("Error", "Please enter a valid email address.")
            return
        
        # Check password match
        if password != confirm:
            messagebox.showerror("Error", "Passwords do not match.")
            return
        
        # Check password strength (minimum 6 characters)
        if len(password) < 6:
            messagebox.showerror("Error", "Password must be at least 6 characters long.")
            return
        
        # Check terms agreement
        if not agree:
            messagebox.showerror("Error", "You must agree to the terms.")
            return

        # Register user in database
        success, message = self.db_manager.register_user(name, email, username, password)
        
        if success:
            messagebox.showinfo("Success", "Account created successfully! You can now log in.")
            self.show_login_page()
        else:
            messagebox.showerror("Registration Failed", message)
    
    def show_text_analyzer(self):
        """Show the main text analyzer interface"""
        self.clear_window()
        self.root.title("🧠 Text Analyzer Tool")
        self.root.geometry("1900x2000")
        self.root.configure(bg="#1e1e2f")
        self.current_page = "analyzer"

        # Canvas for scroll
        canvas = tk.Canvas(self.root, bg="#1e1e2f", highlightthickness=0)
        canvas.pack(side="left", fill="both", expand=True)

        scrollbar_y = tk.Scrollbar(self.root, orient="vertical", command=canvas.yview)
        scrollbar_y.pack(side="right", fill="y")
        canvas.configure(yscrollcommand=scrollbar_y.set)

        content_frame = tk.Frame(canvas, bg="#1e1e2f", width=900)
        content_window = canvas.create_window((0, 0), window=content_frame, anchor="n")

        def resize_canvas(event):
            canvas.itemconfig(content_window, width=event.width)
        canvas.bind("<Configure>", resize_canvas)

        def update_scroll_region(event):
            canvas.configure(scrollregion=canvas.bbox("all"))
        content_frame.bind("<Configure>", update_scroll_region)

        # Header
        tk.Label(content_frame, text="🧠 Text Analyzer Tool", font=("Segoe UI", 26, "bold"),
                bg="#1e1e2f", fg="white").pack(pady=15)

        # Logout Button
        logout_btn = tk.Button(content_frame, text="Logout", font=("Segoe UI", 10), bg="#dc3545", fg="white",
                              padx=20, pady=5, command=self.logout)
        logout_btn.pack(anchor="ne", padx=20)

        # Input Frame
        input_frame = LabelFrame(content_frame, text="Enter Text", font=("Segoe UI", 12, "bold"),
                                bg="#2c2f4c", fg="white", padx=10, pady=10)
        input_frame.pack(padx=20, pady=10, fill="both")
        self.text_input = scrolledtext.ScrolledText(input_frame, height=7, font=("Segoe UI", 13), wrap=tk.WORD,
                                              bg="#3b3f5c", fg="white", insertbackground="white")
        self.text_input.pack(fill="both", expand=True)

        # Analyze Button
        analyze_btn = tk.Button(content_frame, text="🔍 Analyze Text", font=("Segoe UI", 16, "bold"), bg="#3d70f2", fg="white",
                 padx=30, pady=10, command=self.analyze_text)
        analyze_btn.pack(pady=10)

        # Export PDF Button (initially hidden)
        self.export_btn = tk.Button(content_frame, text="📄 Export as PDF", font=("Segoe UI", 14, "bold"), 
                                   bg="#28a745", fg="white", padx=30, pady=8, command=self.export_pdf)
        # Initially pack but make invisible
        self.export_btn.pack(pady=5)
        self.export_btn.pack_forget()  # Hide initially

        # Output Frame
        output_frame = LabelFrame(content_frame, text="Analysis Result", font=("Segoe UI", 12, "bold"),
                                 bg="#2c2f4c", fg="white", padx=10, pady=10)
        output_frame.pack(padx=20, pady=10, fill="both", expand=True)
        self.output_text = scrolledtext.ScrolledText(output_frame, height=12, font=("Consolas", 13), wrap=tk.WORD,
                                               bg="#1e1e2f", fg="#00ffcc", insertbackground="white", state="disabled")
        self.output_text.pack(fill="both", expand=True)

        # Graph Frame
        graph_frame = LabelFrame(content_frame, text="📊 Analysis Graphs", font=("Segoe UI", 12, "bold"),
                                bg="#1e1e2f", fg="white", bd=0, padx=10, pady=10)
        graph_frame.pack(padx=30, pady=10, fill="both", expand=True)

        graph_canvas = tk.Canvas(graph_frame, bg="#1e1e2f", highlightthickness=0)
        graph_canvas.pack(side="left", fill="both", expand=True)

        graph_scrollbar_y = tk.Scrollbar(graph_frame, orient="vertical", command=graph_canvas.yview)
        graph_scrollbar_y.pack(side="right", fill="y")
        graph_scrollbar_x = tk.Scrollbar(graph_frame, orient="horizontal", command=graph_canvas.xview)
        graph_scrollbar_x.pack(side="bottom", fill="x")

        graph_canvas.configure(yscrollcommand=graph_scrollbar_y.set, xscrollcommand=graph_scrollbar_x.set)

        graph_inner_frame = tk.Frame(graph_canvas, bg="#1e1e2f")
        graph_canvas.create_window((0, 0), window=graph_inner_frame, anchor="nw")

        def update_graph_scroll(event):
            graph_canvas.configure(scrollregion=graph_canvas.bbox("all"))
        graph_inner_frame.bind("<Configure>", update_graph_scroll)

        self.graph_label = tk.Label(graph_inner_frame, bg="#1e1e2f")
        self.graph_label.pack()
    
    def logout(self):
        """Handle user logout"""
        response = messagebox.askyesno("Logout", "Are you sure you want to logout?")
        if response:
            self.show_login_page()
    
    def analyze_text(self):
        """Analyze the input text"""
        input_text = self.text_input.get("1.0", tk.END).strip()

        if not input_text:
            messagebox.showwarning("⚠️ Warning", "Please enter some text!")
            return

        blob = TextBlob(input_text)
        sentiment = blob.sentiment
        keywords = blob.noun_phrases
        tokens = blob.words

        # Store analysis results for PDF export
        self.analysis_results = {
            'input_text': input_text,
            'sentiment': sentiment,
            'keywords': keywords,
            'tokens': tokens,
            'analysis_date': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        result = f"🧠 TEXT ANALYSIS RESULT\n\n"
        result += "📊 SENTIMENT ANALYSIS\n"
        result += f"   ➔ Polarity     : {sentiment.polarity:.2f}\n"
        result += f"   ➔ Subjectivity : {sentiment.subjectivity:.2f}\n\n"

        result += "🔑 KEYWORDS\n"
        result += "\n".join([f"   {i+1}. {kw}" for i, kw in enumerate(keywords)]) + "\n" if keywords else "   No significant keywords found.\n"

        result += f"\n🤩 TOKENS ({len(tokens)} words)\n"
        result += "   " + ", ".join(tokens)

        self.output_text.config(state="normal")
        self.output_text.delete("1.0", tk.END)
        self.output_text.insert(tk.END, result)
        self.output_text.config(state="disabled")

        # Show export button after analysis
        self.export_btn.pack(pady=5)

        self.show_analysis_graphs(input_text)
    
    def show_analysis_graphs(self, text):
        """Generate and display analysis graphs"""
        blob = TextBlob(text)
        sentiment = blob.sentiment
        tokens = [word.lower() for word in blob.words if word.isalpha()]
        token_counts = Counter(tokens).most_common(10)

        fig, axs = plt.subplots(1, 2, figsize=(10, 4), dpi=100)
        fig.patch.set_facecolor('#1e1e2f')

        axs[0].bar(["Polarity", "Subjectivity"], [sentiment.polarity, sentiment.subjectivity],
                  color=["#3498db", "#9b59b6"])
        axs[0].set_title("Sentiment Score", color="white")
        axs[0].tick_params(colors='white')
        axs[0].set_ylim(-1, 1)
        for spine in axs[0].spines.values():
            spine.set_color('white')

        if token_counts:
            labels, values = zip(*token_counts)
            axs[1].barh(labels, values, color="#f39c12")
            axs[1].set_title("Top 10 Tokens", color="white")
            axs[1].tick_params(colors='white')
            axs[1].invert_yaxis()
            for spine in axs[1].spines.values():
                spine.set_color('white')

        buf = io.BytesIO()
        plt.tight_layout()
        plt.savefig(buf, format='png', facecolor=fig.get_facecolor())
        buf.seek(0)
        img = Image.open(buf)
        img_tk = ImageTk.PhotoImage(img)

        self.graph_label.config(image=img_tk)
        self.graph_label.image = img_tk
        buf.close()
        plt.close()

    def export_pdf(self):
        """Export analysis results to PDF"""
        if not hasattr(self, 'analysis_results'):
            messagebox.showwarning("No Analysis", "Please analyze some text first!")
            return

        try:
            # Ask user where to save the PDF
            filename = filedialog.asksaveasfilename(
                defaultextension=".pdf",
                filetypes=[("PDF files", "*.pdf"), ("All files", "*.*")],
                title="Save Analysis Report"
            )
            
            if not filename:
                return  # User cancelled

            # Create PDF document
            doc = SimpleDocTemplate(filename, pagesize=A4, rightMargin=72, leftMargin=72,
                                  topMargin=72, bottomMargin=18)

            # Get styles
            styles = getSampleStyleSheet()
            title_style = ParagraphStyle(
                'CustomTitle',
                parent=styles['Heading1'],
                fontSize=24,
                spaceAfter=30,
                alignment=1,  # Center alignment
                textColor=colors.darkblue
            )
            
            heading_style = ParagraphStyle(
                'CustomHeading',
                parent=styles['Heading2'],
                fontSize=16,
                spaceAfter=12,
                textColor=colors.darkgreen
            )
            
            normal_style = styles['Normal']
            normal_style.fontSize = 12

            # Build PDF content
            story = []

            # Title
            story.append(Paragraph("📊 Text Analysis Report", title_style))
            story.append(Spacer(1, 20))

            # Analysis Date
            story.append(Paragraph(f"<b>Analysis Date:</b> {self.analysis_results['analysis_date']}", normal_style))
            story.append(Spacer(1, 20))

            # Original Text Section
            story.append(Paragraph("📝 Original Text", heading_style))
            # Limit text display to first 500 characters for readability
            display_text = self.analysis_results['input_text']
            if len(display_text) > 500:
                display_text = display_text[:500] + "..."
            story.append(Paragraph(display_text, normal_style))
            story.append(Spacer(1, 20))

            # Sentiment Analysis Section
            story.append(Paragraph("📊 Sentiment Analysis", heading_style))
            
            sentiment_data = [
                ['Metric', 'Value', 'Interpretation'],
                ['Polarity', f"{self.analysis_results['sentiment'].polarity:.3f}", 
                 self.get_polarity_interpretation(self.analysis_results['sentiment'].polarity)],
                ['Subjectivity', f"{self.analysis_results['sentiment'].subjectivity:.3f}",
                 self.get_subjectivity_interpretation(self.analysis_results['sentiment'].subjectivity)]
            ]
            
            sentiment_table = Table(sentiment_data, colWidths=[1.5*inch, 1*inch, 3*inch])
            sentiment_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, 0), 12),
                ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                ('GRID', (0, 0), (-1, -1), 1, colors.black)
            ]))
            
            story.append(sentiment_table)
            story.append(Spacer(1, 20))

            # Keywords Section
            story.append(Paragraph("🔑 Key Phrases", heading_style))
            if self.analysis_results['keywords']:
                keywords_text = "<br/>".join([f"• {kw}" for kw in self.analysis_results['keywords'][:10]])  # Limit to top 10
            else:
                keywords_text = "No significant key phrases found."
            story.append(Paragraph(keywords_text, normal_style))
            story.append(Spacer(1, 20))

            # Text Statistics Section
            story.append(Paragraph("📈 Text Statistics", heading_style))
            
            word_count = len(self.analysis_results['tokens'])
            char_count = len(self.analysis_results['input_text'])
            sentence_count = len(TextBlob(self.analysis_results['input_text']).sentences)
            
            stats_data = [
                ['Statistic', 'Count'],
                ['Total Words', str(word_count)],
                ['Total Characters', str(char_count)],
                ['Total Sentences', str(sentence_count)],
                ['Average Words per Sentence', f"{word_count/max(sentence_count, 1):.1f}"]
            ]
            
            stats_table = Table(stats_data, colWidths=[3*inch, 1.5*inch])
            stats_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.darkblue),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, 0), 12),
                ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                ('BACKGROUND', (0, 1), (-1, -1), colors.lightblue),
                ('GRID', (0, 0), (-1, -1), 1, colors.black)
            ]))
            
            story.append(stats_table)
            story.append(Spacer(1, 20))

            # Top Words Section (if available)
            if self.analysis_results['tokens']:
                story.append(Paragraph("📝 Most Frequent Words", heading_style))
                tokens = [word.lower() for word in self.analysis_results['tokens'] if word.isalpha()]
                word_freq = Counter(tokens).most_common(10)
                
                if word_freq:
                    freq_data = [['Word', 'Frequency']] + [[word, str(count)] for word, count in word_freq]
                    freq_table = Table(freq_data, colWidths=[2*inch, 1*inch])
                    freq_table.setStyle(TableStyle([
                        ('BACKGROUND', (0, 0), (-1, 0), colors.darkgreen),
                        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                        ('FONTSIZE', (0, 0), (-1, 0), 12),
                        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                        ('BACKGROUND', (0, 1), (-1, -1), colors.lightgreen),
                        ('GRID', (0, 0), (-1, -1), 1, colors.black)
                    ]))
                    story.append(freq_table)

            # Footer
            story.append(Spacer(1, 30))
            footer_style = ParagraphStyle(
                'Footer',
                parent=styles['Normal'],
                fontSize=10,
                alignment=1,
                textColor=colors.grey
            )
            story.append(Paragraph("Generated by TextAnalyzer Tool", footer_style))

            # Build PDF
            doc.build(story)
            
            messagebox.showinfo("Export Successful", f"Analysis report saved successfully!\nLocation: {filename}")

        except Exception as e:
            messagebox.showerror("Export Error", f"Failed to export PDF: {str(e)}")

    def get_polarity_interpretation(self, polarity):
        """Get interpretation for polarity score"""
        if polarity > 0.1:
            return "Positive sentiment"
        elif polarity < -0.1:
            return "Negative sentiment"
        else:
            return "Neutral sentiment"

    def get_subjectivity_interpretation(self, subjectivity):
        """Get interpretation for subjectivity score"""
        if subjectivity > 0.6:
            return "Highly subjective"
        elif subjectivity > 0.3:
            return "Moderately subjective"
        else:
            return "Mostly objective"

    def __del__(self):
        """Destructor to close database connection"""
        if hasattr(self, 'db_manager'):
            self.db_manager.close_connection()

# Create and start the application
if __name__ == "__main__":
    app = TextAnalyzerApp()
    app.start_app()

Successfully connected to MySQL database
Users table created successfully
MySQL connection closed


In [1]:
pip install reportlab

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ------------------------------------- -- 1.8/2.0 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 5.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
